# WK - Projekt 1
### Adam Wiatrowski, 148510
### Maciej Wieczorek, 148141

In [ ]:
import cv2
import numpy as np
from pathlib import Path
import PIL
from ipywidgets import widgets, interact
from IPython.display import display, clear_output
from skimage.exposure import rescale_intensity

In [ ]:
def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(PIL.Image.fromarray(a))

In [ ]:
def imshow_pixel(p):
    imshow(np.full((64, 64, 3), p))

# Katedra

In [ ]:
DATA_DIR = 'cathedral'
imgs = []
imgs_gray = []
for img_path in Path(DATA_DIR).iterdir(): 
    imgs.append(cv2.imread(str(img_path)))
    imgs_gray.append(cv2.imread(str(img_path), 0))

imgs = np.array(imgs)
imgs_gray = np.array(imgs_gray)

### Uśrednianie

In [ ]:
imshow(np.average(imgs, axis=0))

### Mediana

In [ ]:
imshow(np.median(imgs, axis=0))

### Odległość od globlnego średniego koloru

In [ ]:
avg_color = np.average(imgs.reshape(-1, 3), axis=0)
imshow_pixel(avg_color)

In [ ]:
result1 = np.zeros_like(imgs[0])
for i in range(imgs.shape[1]):
    for j in range(imgs.shape[2]):
        k = np.argmin(np.linalg.norm(imgs[:,i,j] - avg_color, axis=1))
        result1[i][j] = imgs[k][i][j]

imshow(result1)

In [ ]:
imshow_pixel(result1[415,319])
print(result1[415,319]) # duża odległość na trzecim kanale
print(avg_color)

### Minimalna z maksymalnych odległości każdego kanału od globalnego średniego koloru

In [ ]:
result2 = np.zeros_like(imgs[0])
for i in range(imgs.shape[1]):
    for j in range(imgs.shape[2]):
        k = np.argmin(np.max(np.abs(imgs[:,i,j] - avg_color), axis=1))
        result2[i][j] = imgs[k][i][j]

imshow(result2)

In [ ]:
# dodadkowa mała poprawa z wykorzystaniem erozji
struct = np.ones([2, 2], np.uint8)
imshow(cv2.erode(result2, struct, iterations=1))

# Oświetlenie

In [ ]:
DATA_DIR = 'bronze'
imgs = []
imgs_gray = []
for img_path in Path(DATA_DIR).iterdir():
  if(str(img_path)[-3:] == "JPG"):
    imgs.append(cv2.imread(str(img_path)))
    imgs_gray.append(cv2.imread(str(img_path), 0))

In [ ]:
# Wczytanie obrazu końcowego ze zbioru zadań

target = cv2.imread("result.png")
target_gray = cv2.imread("result.png", 0)

imshow(target)
imshow(target_gray)

In [ ]:
# Mediana
result_median = np.median(imgs, axis=0)

# Średnia
result_mean = np.mean(imgs, axis=0)

imshow(np.concatenate([result_median, result_mean], 1))

In [ ]:
# Max
result_max = np.max(imgs, axis=0)

# Min
result_min = np.min(imgs, axis=0)


imshow(np.concatenate([result_max, result_min], 1))

### Podzielenie obrazów ze względu na ich stopnie jasności

In [ ]:
results = np.zeros_like(imgs)

g_copy = np.copy(imgs_gray)
for im in range(results.shape[0]):
  for i in range(results.shape[1]):
    for j in range(results.shape[2]):
      k = np.argmax([x[i][j] for x in g_copy])
      results[im][i][j] = imgs[k][i][j]
      g_copy[k][i][j] = 0

In [ ]:
#Spadek stopnia jasności wraz z kolejnymi obrazami

imshow(np.concatenate(results[0:3], 1))

### Maska pozwalająca na uzyskanie średniego, stałego tła


In [ ]:
mean_background = np.mean(imgs_gray, axis=0).astype(np.uint16)
mask = 185 <= mean_background

mask_rgb = np.stack([mask] * 3, axis=-1)
imshow(mean_background)

### Interakcja z użytkownikiem - wybór odpowiedniego stopnia naświetlenia

In [ ]:
def selective_light(i):
  masked = np.where((1 - mask_rgb), results[i], result_mean)

  imshow(masked)

In [ ]:
# Interakcja z użytkownikiem


slider = widgets.IntSlider(min = 0, max = len(imgs)-1)

button = widgets.Button(description="Show Image")

def on_button_click(b):
    val = slider.value
    clear_output()
    display(slider)
    display(button)
    selective_light(val)


button.on_click(on_button_click)

display(slider)
display(button)

In [ ]:
# Zastosowanie maski w celu wyodrębnienia figury
imshow(np.concatenate(imgs[0:3]*(1 - mask_rgb), 1))

imshow(np.concatenate(imgs[0:3]*(mask_rgb), 1))

In [ ]:
# Wykrycie krawędzi, wykorzystanie operacji splotowych

g = np.array([
  [0, -1, 0],
  [-1, 4, -1],
  [0, -1, 0]
], np.float32)

img_grayscale = np.copy(mean_background) / 255.0

img_edges = cv2.filter2D(img_grayscale, -1, g)
imshow(np.concatenate([img_grayscale, img_edges], 1) * 255.0)

img_edges = np.abs(img_edges)
img_edges = rescale_intensity(img_edges)

imshow(np.concatenate([img_grayscale, img_edges], 1) * 255.0)

In [ ]:
# Operacje morfologiczne

# dylatacja

struct = np.ones((1, 1),np.uint8)

img_morf = (target_gray > 175).astype(np.uint8) * 255
img_bin_dil = cv2.dilate(img_morf, struct, iterations=1)

imshow(img_bin_dil)

# zamknięcie

img_edge_close = cv2.morphologyEx(img_morf, cv2.MORPH_CLOSE, struct)
img_edge_open = cv2.morphologyEx(img_edge_close, cv2.MORPH_OPEN, struct)
img_edge_close2 = cv2.morphologyEx(img_edge_open, cv2.MORPH_CLOSE, struct)
img_space_Q = np.minimum(img_morf, img_edge_close2)

imshow(np.concatenate([img_edge_close, img_edge_open, img_edge_close2], 1))

imshow(np.concatenate([img_morf, img_space_Q], 1))

In [ ]:
# Stworzenie maski w celu odwzrowoania obrazu referencyjnego (result.png)

new_mask = img_morf*(1 - mask) == 255

new_mask_rgb = np.stack([new_mask] * 3, axis=-1)

masked_new = np.where(new_mask_rgb, result_max, imgs[0])
result = np.where((1 - mask_rgb), masked_new, result_mean)


imshow(result)

# Maska
imshow(result_max*(new_mask_rgb))